In [1]:
import gzip
import zipfile
import json
from collections import defaultdict
import math
import scipy.optimize
from sklearn import svm
import numpy as np
import pandas as pd
import string
import random
import string
from sklearn import linear_model
import matplotlib.pyplot as plt
from implicit.bpr import BayesianPersonalizedRanking
from implicit.als import AlternatingLeastSquares
from sklearn.model_selection import train_test_split
from scipy.sparse import coo_matrix, csr_matrix

In [2]:
import json
 
filter_path = 'data/filter_all_t.json'
 
# Open and read the entire content of the JSON file
with open(filter_path, 'r') as json_file:
    filter_data_single_line = json_file.read()
 
# Parse the JSON data
try:
    json_filter_data = json.loads(filter_data_single_line)
    # Now you can work with the parsed JSON data
    print(json_filter_data)
except json.JSONDecodeError as err:
    print(f'Error decoding JSON: {err}')
    

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [3]:
imagerev_path = 'data/image_review_all.json'

In [4]:
def getjson2(fname, maxno=-1):
    data_list = list()
    no = 0
    with open(fname, 'r') as json_file:
        for line_number, line in enumerate(json_file,1):
            try:
                imagerev_data = json.loads(line)
                data_list.append(imagerev_data)
                no += 1
                if (maxno > -1) & (no >= maxno):
                    break
            except json.JSONDecodeError as err:
                print(f'Error decoding JSON: {err}')
        return data_list

In [5]:
data = getjson2(imagerev_path)

# Baseline

In [6]:
df = pd.DataFrame(data)

In [7]:
business_means = df.groupby('business_id')['rating'].mean()

In [8]:
ids = df['business_id']

In [9]:
preds = [round(business_means[id]) for id in ids]
y = df['rating']

In [10]:
correct = preds == y
accuracy = sum(correct) / len(correct)

In [11]:
print(accuracy)

0.5489589291727693


# Model

In [12]:
users = df.groupby('user_id')[['rating']].mean().reset_index().reset_index().set_index('user_id')

In [13]:
df['u_index'] = df['user_id'].apply(lambda x: users.loc[x]['index'].astype(np.int32))

In [14]:
businesses = pd.DataFrame(business_means).reset_index().reset_index().set_index('business_id')

In [15]:
df['b_index'] = df['business_id'].apply(lambda x: businesses.loc[x]['index'].astype(np.int32))

In [50]:
#split data
train, test = train_test_split(df, test_size=0.25, random_state=100)

#create sparse matrix
matrix = csr_matrix(
    (train['rating'].astype(np.float32),
    (train['u_index'], train['b_index'])),
    shape=(len(users), len(businesses)
))

In [32]:
model = AlternatingLeastSquares()
model.fit(matrix)

/Users/William/anaconda3/lib/python3.11/site-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 8 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()


  0%|          | 0/15 [00:00<?, ?it/s]

In [33]:
user_indices = test['u_index'].unique()
business_indices = test['b_index'].unique()

In [44]:
matrix.T

<64527x868937 sparse matrix of type '<class 'numpy.float32'>'
	with 1115810 stored elements in Compressed Sparse Column format>

In [55]:
matrix

<868937x64527 sparse matrix of type '<class 'numpy.float32'>'
	with 1115810 stored elements in Compressed Sparse Row format>

In [54]:
N = len(business_indices)
recommendations = model.recommend(user_indices[0], user_items=matrix.T, N=N)

predicted_business_indices, predicted_scores = zip(*recommendations)

predicted_ratings = np.zeros_like(business_indices, dtype=float)

predicted_ratings[np.array(predicted_business_indices)] = predicted_scores

ValueError: user_items needs to be a CSR sparse matrix

In [ ]:
preds = predictions.flatten()

In [ ]:
actual = test_matrix.data

In [45]:
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split
from surprise import SVD
from surprise import accuracy
from surprise.model_selection import GridSearchCV

In [16]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[['user_id', 'business_id', 'rating']], reader)

trainset, testset = train_test_split(data, test_size=0.25, random_state=100)

In [17]:
model = SVD()
model.fit(trainset)

In [18]:
predictions = model.test(testset)

In [19]:
mse = accuracy.mse(predictions)
print(f'Mean Squared Error: {mse}')

MSE: 0.8988
Mean Squared Error: 0.8988260265913174


In [22]:
preds_ratings = [round(pred.est) for pred in predictions]

In [54]:
test_ratings = np.array([x[2] for x in testset])

In [41]:
correct = np.array(preds_ratings) == np.array(test_ratings)
accuracy = sum(correct) / len(correct)

In [42]:
print(accuracy)

0.5376582593288648
